In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as sns

In [ ]:
df = pd.read_excel("ng_solar_dataset_10000 - Copy.xlsx")

# 1. Initial Data Exploration

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

# 2. Data Cleaning

## 2.1 Handling Missing Values

First, let's check which columns have missing values and how many.

In [ ]:
df.isnull().sum()

There are two main strategies for dealing with missing values:1. **Dropping:** Removing the rows or columns with missing values. This is simple but can lead to significant data loss.2. **Imputation:** Filling in the missing values with a calculated value (e.g., mean, median, mode). This preserves the data but adds artificial values.\nWe will proceed with imputation, as it avoids data loss. We will impute numerical columns with the median (robust to outliers) and categorical columns with the mode (most frequent value).

In [ ]:
for col in df.columns:    if df[col].dtype == 'object':        # Impute categorical columns with mode        df[col].fillna(df[col].mode()[0], inplace=True)    else:        # Impute numerical columns with median        df[col].fillna(df[col].median(), inplace=True)

Now, let's verify that there are no more missing values.

In [ ]:
print(f"Total missing values after imputation: {df.isnull().sum().sum()}")

## 2.2 Handling Duplicate Entries

First, let's check for any duplicate rows in the dataset.

In [ ]:
print(f"Number of duplicate rows: {df.duplicated().sum()}")

Now, let's remove them and reset the index.

In [ ]:
df.drop_duplicates(inplace=True)df.reset_index(drop=True, inplace=True)

Finally, verify that no duplicates remain.

In [ ]:
print(f"Number of duplicate rows after cleaning: {df.duplicated().sum()}")

## 2.3 Cleaning Text Data (Inconsistencies & Typos)

Text data can be messy. Common issues include inconsistent capitalization and extra whitespace. We can clean this up by converting all text to lowercase and stripping whitespace from the beginning and end of strings.

In [ ]:
text_cols = df.select_dtypes(include=['object']).columnsprint(f"Text columns identified: {list(text_cols)}")

Let's look at a sample of the unique values in the first text column before cleaning. Pay attention to any mixed case or extra spaces.

In [ ]:
if len(text_cols) > 0:    print(f"Unique values in '{text_cols[0]}' before cleaning (sample):")    print(df[text_cols[0]].unique()[:5])

Now, we apply the cleaning transformations to all text columns.

In [ ]:
for col in text_cols:    df[col] = df[col].str.lower()    df[col] = df[col].str.strip()

Let's look at the same column again after cleaning to see the effect. All text should now be lowercase and free of leading/trailing whitespace.

In [ ]:
if len(text_cols) > 0:    print(f"Unique values in '{text_cols[0]}' after cleaning (sample):")    print(df[text_cols[0]].unique()[:5])

## 2.4 Cleaning Numerical Data (Out-of-Range & Outliers)

### 2.4.1 Handling Incorrect or Out-of-Range ValuesSome data may contain values that are not logically possible. For example, in a solar dataset, we wouldn't expect to see negative values for measurements like solar radiation. We will check for and correct such values.

In [ ]:
numeric_cols = df.select_dtypes(include=np.number).columnsprint(f"Checking for negative values in numerical columns: {list(numeric_cols)}")for col in numeric_cols:    negative_count = (df[col] < 0).sum()    print(f"- Found {negative_count} negative values in '{col}'.")

A common strategy for such values is to cap them at a valid minimum (like 0). This is called 'clipping'.

In [ ]:
for col in numeric_cols:    df[col] = df[col].clip(lower=0)print("Clipped all negative values in numerical columns to 0.")

### 2.4.2 Handling OutliersOutliers are data points that are significantly different from other observations. They can skew statistical analysis. We will use the Interquartile Range (IQR) method to detect them.An outlier is a data point that falls outside of `1.5 * IQR` below the first quartile (Q1) or above the third quartile (Q3).

In [ ]:
print("Identifying outliers using the IQR method:")for col in numeric_cols:    Q1 = df[col].quantile(0.25)    Q3 = df[col].quantile(0.75)    IQR = Q3 - Q1    lower_bound = Q1 - 1.5 * IQR    upper_bound = Q3 + 1.5 * IQR    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]    print(f"- Found {len(outliers)} outliers in '{col}'.")

Instead of removing outliers, which can cause data loss, we can cap them. This means any value below the lower bound is set to the lower bound, and any value above the upper bound is set to the upper bound. This is also known as 'winsorizing'.

In [ ]:
for col in numeric_cols:    Q1 = df[col].quantile(0.25)    Q3 = df[col].quantile(0.75)    IQR = Q3 - Q1    lower_bound = Q1 - 1.5 * IQR    upper_bound = Q3 + 1.5 * IQR    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)print("Capped all outliers in numerical columns.")

Let's verify that the outliers have been handled.

In [ ]:
print("Re-checking for outliers after capping:")for col in numeric_cols:    Q1 = df[col].quantile(0.25)    Q3 = df[col].quantile(0.75)    IQR = Q3 - Q1    lower_bound = Q1 - 1.5 * IQR    upper_bound = Q3 + 1.5 * IQR    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]    print(f"- Found {len(outliers)} outliers in '{col}'.")

## 2.5 Correcting Data Types

Ensuring each column has the correct data type is crucial for analysis and memory efficiency. For example, a column of numbers stored as text (object type) can't be used in mathematical calculations.

Let's look at the current data types.

In [ ]:
df.info()

We can use `pd.to_numeric()` to convert columns that should be numerical and `pd.to_datetime()` for date/time columns. Below is a demonstration.We use `errors='coerce'`, which will turn any value that cannot be converted into `NaN` (Not a Number). We then have to re-impute these new missing values.

In [ ]:
for col in df.select_dtypes(include=['object']).columns:    # Attempt to convert object columns to numeric, coercing errors    # This is a general approach; in a real scenario, you'd target specific columns.    df[col] = pd.to_numeric(df[col], errors='coerce')        # Re-impute any NaNs created by the coercion    if df[col].isnull().sum() > 0:        df[col].fillna(df[col].median(), inplace=True)# Example for converting a date column (replace 'your_date_column' with a real one if it exists)# if 'your_date_column' in df.columns:#     df['your_date_column'] = pd.to_datetime(df['your_date_column'], errors='coerce')#     # For datetime, we might fill with the mode or a specific placeholder#     if df['your_date_column'].isnull().sum() > 0:#         df['your_date_column'].fillna(df['your_date_column'].mode()[0], inplace=True)

Let's check the data types again to confirm the changes.

In [ ]:
df.info()

## 2.6 Handling Data Merging Issues

When working on a real-world project, you often need to combine data from multiple sources. This is done by 'merging' or 'joining' datasets. Pandas provides the powerful `pd.merge()` function for this.\nKey concepts for merging:- **Keys:** The column(s) that the DataFrames have in common and are used to match rows. For example, a `user_id` column.- **Join Type:** How to handle rows that don't have a matching key in the other DataFrame.    - `inner` (default): Keep only the rows where the key exists in **both** DataFrames.    - `outer`: Keep **all** rows from both DataFrames, filling in `NaN` where there is no match.    - `left`: Keep all rows from the **left** DataFrame, and only the matching rows from the right.    - `right`: Keep all rows from the **right** DataFrame, and only the matching rows from the left.

Since we only have one dataset here, we can't perform a real merge. However, the code cell below provides a template for how you would do it.

In [ ]:
# Assume you have a second DataFrame called 'df2' that you've loaded# and it shares a common column, e.g., 'location_id', with our main 'df'.# Example DataFrames (uncomment to run)# data1 = {'location_id': [1, 2, 3], 'temperature': [35, 32, 34]}# df1 = pd.DataFrame(data1)# print("DataFrame 1:")# print(df1)# # data2 = {'location_id': [1, 2, 4], 'manager': ['Ali', 'Binta', 'Charles']}# df2 = pd.DataFrame(data2)# print("\nDataFrame 2:")# print(df2)# --- INNER JOIN ---# merged_inner = pd.merge(df1, df2, on='location_id', how='inner')# print("\nInner Join Result:")# print(merged_inner) # Will only have location_id 1 and 2# --- LEFT JOIN ---# merged_left = pd.merge(df1, df2, on='location_id', how='left')# print("\nLeft Join Result:")# print(merged_left) # Will have all of df1's rows

# 3. Final Verification

After all the cleaning steps, let's perform a final check to see the state of our data.

In [ ]:
print("--- Final Data Quality Report ---")print(f"Total Missing Values: {df.isnull().sum().sum()}")print(f"Total Duplicate Rows: {df.duplicated().sum()}")print(f"Final DataFrame Shape: {df.shape}")print("---------------------------------")

Let's also look at the first few rows of the fully cleaned DataFrame.

In [ ]:
df.head()